In [1]:
import os
import cv2
import numpy as np
from deepface import DeepFace
from mtcnn import MTCNN
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt


In [2]:
detector = MTCNN()


Exception ignored in: <_io.BufferedReader>
Traceback (most recent call last):
  File "G:\anaconda\Lib\site-packages\lz4\frame\__init__.py", line 753, in flush
    self._fp.flush()
ValueError: I/O operation on closed file.
Exception ignored in: <_io.BufferedReader>
Traceback (most recent call last):
  File "G:\anaconda\Lib\site-packages\lz4\frame\__init__.py", line 753, in flush
    self._fp.flush()
ValueError: I/O operation on closed file.
Exception ignored in: <_io.BufferedReader>
Traceback (most recent call last):
  File "G:\anaconda\Lib\site-packages\lz4\frame\__init__.py", line 753, in flush
    self._fp.flush()
ValueError: I/O operation on closed file.


In [3]:
def extract_face(image_path, size=(160,160), margin=20):
    img = cv2.imread(image_path)
    if img is None:
        return None

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    faces = detector.detect_faces(img)

    if len(faces) == 0:
        return None

    x, y, w, h = faces[0]["box"]
    x, y = abs(x), abs(y)

    h_img, w_img, _ = img.shape
    x1 = max(x - margin, 0)
    y1 = max(y - margin, 0)
    x2 = min(x + w + margin, w_img)
    y2 = min(y + h + margin, h_img)

    face = img[y1:y2, x1:x2]
    face = cv2.resize(face, size)

    return face


In [4]:
def get_embedding(face):
    embedding = DeepFace.represent(
        face,
        model_name="ArcFace",
        enforce_detection=False
    )[0]["embedding"]

    return np.array(embedding)


In [5]:
DATASET_DIR = r"C:\Users\umara\Face Recognition\Face Recongnized\Peoples"

X_embeddings = []
y_labels = []

total = 0

for person in os.listdir(DATASET_DIR):
    person_path = os.path.join(DATASET_DIR, person)
    if not os.path.isdir(person_path):
        continue

    print("Processing:", person)

    for img_name in os.listdir(person_path):
        if not img_name.lower().endswith((".jpg", ".jpeg", ".png")):
            continue

        img_path = os.path.join(person_path, img_name)
        face = extract_face(img_path)

        if face is None:
            continue

        emb = get_embedding(face)

        X_embeddings.append(emb)
        y_labels.append(person)
        total += 1

print("Total images trained:", total)
print("Total persons:", len(set(y_labels)))


Processing: Aaron_Eckhart
Processing: Aaron_Guiel
Processing: Aaron_Patterson
Processing: Aaron_Peirsol
Processing: Aaron_Pena
Processing: Aaron_Sorkin
Processing: Aaron_Tippin
Processing: Abbas_Kiarostami
Processing: Abba_Eban
Processing: Abdel_Aziz_Al-Hakim
Processing: Abdel_Madi_Shabneh
Processing: Abdel_Nasser_Assidi
Processing: Abdoulaye_Wade
Processing: Abdulaziz_Kamilov
Processing: Abdullah
Processing: Abdullah_Ahmad_Badawi
Processing: Abdullah_al-Attiyah
Processing: Abdullah_Gul
Processing: Abdullah_Nasseef
Processing: Abdullatif_Sener
Processing: Abdul_Majeed_Shobokshi
Processing: Abdul_Rahman
Processing: Abel_Aguilar
Processing: Abel_Pacheco
Processing: Abid_Hamid_Mahmud_Al-Tikriti
Processing: Abner_Martinez
Processing: Abraham_Foxman
Processing: Aby_Har-Even
Processing: Adam_Ant
Processing: Adam_Freier
Processing: Adam_Herbert
Processing: Adam_Kennedy
Processing: Adam_Mair
Processing: Adam_Rich
Processing: Adam_Sandler
Processing: Adam_Scott
Processing: Adelina_Avila
Process

In [6]:
X_embeddings = np.array(X_embeddings)
y_labels = np.array(y_labels)

np.save("X_embeddings.npy", X_embeddings)
np.save("y_labels.npy", y_labels)

print("✅ Embeddings saved")


✅ Embeddings saved


In [2]:
import numpy as np
X_embeddings = np.load("X_embeddings.npy")
y_labels = np.load("y_labels.npy")

print(X_embeddings.shape, y_labels.shape)


(13235, 512) (13235,)


In [10]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Convert lists to arrays
X = np.array(X_embeddings)  # shape: (num_images, 512)
y = np.array(y_labels)      # shape: (num_images,)

# Create KNN classifier
knn = KNeighborsClassifier(n_neighbors=1, metric='cosine')  # cosine similarity

# Train on all embeddings
knn.fit(X, y)

# Predict on the same training data (closed-set)
y_pred = knn.predict(X)

# Calculate accuracy
accuracy = accuracy_score(y, y_pred) * 100
print(f"Training accuracy (closed-set): {accuracy:.2f}%")


Training accuracy (closed-set): 100.00%


In [8]:
def recognize_face(embedding, X_embeddings, y_labels, threshold=0.6):
    similarities = cosine_similarity([embedding], X_embeddings)[0]
    best_idx = np.argmax(similarities)
    best_score = similarities[best_idx]

    if best_score >= threshold:
        return y_labels[best_idx], best_score
    else:
        return "Unknown", best_score


In [2]:
import gradio as gr
import cv2
import numpy as np
from mtcnn import MTCNN
from deepface import DeepFace
from sklearn.metrics.pairwise import cosine_similarity


In [3]:
detector = MTCNN()
X_embeddings = np.load("X_embeddings.npy")
y_labels = np.load("y_labels.npy")


Exception ignored in: <_io.BufferedReader>
Traceback (most recent call last):
  File "G:\anaconda\Lib\site-packages\lz4\frame\__init__.py", line 753, in flush
    self._fp.flush()
ValueError: I/O operation on closed file.
Exception ignored in: <_io.BufferedReader>
Traceback (most recent call last):
  File "G:\anaconda\Lib\site-packages\lz4\frame\__init__.py", line 753, in flush
    self._fp.flush()
ValueError: I/O operation on closed file.
Exception ignored in: <_io.BufferedReader>
Traceback (most recent call last):
  File "G:\anaconda\Lib\site-packages\lz4\frame\__init__.py", line 753, in flush
    self._fp.flush()
ValueError: I/O operation on closed file.


In [4]:
def extract_face(img_array, size=(160,160), margin=20):
    img_rgb = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
    faces = detector.detect_faces(img_rgb)
    if len(faces) == 0:
        return None

    x, y, w, h = faces[0]['box']
    x, y = abs(x), abs(y)
    h_img, w_img, _ = img_rgb.shape

    x1 = max(x - margin, 0)
    y1 = max(y - margin, 0)
    x2 = min(x + w + margin, w_img)
    y2 = min(y + h + margin, h_img)

    face = img_rgb[y1:y2, x1:x2]
    face = cv2.resize(face, size)
    return face


In [5]:
def get_embedding(face):
    return np.array(
        DeepFace.represent(face, model_name="ArcFace", enforce_detection=False)[0]["embedding"]
    )


In [6]:
def recognize_face(embedding, threshold=0.6):
    sims = cosine_similarity([embedding], X_embeddings)[0]
    best_idx = np.argmax(sims)
    best_score = sims[best_idx]

    if best_score >= threshold:
        return y_labels[best_idx], best_score
    else:
        return "Unknown", best_score


In [7]:
def gradio_recognize_user(image):
    img_array = np.array(image)
    face = extract_face(img_array)
    if face is None:
        return None, "❌ No face detected!"

    emb = get_embedding(face)
    name, score = recognize_face(emb, threshold=0.6)

    return face, f"{name} | Confidence: {score*100:.2f}%"


In [8]:
gr.Interface(
    fn=gradio_recognize_user,
    inputs=gr.Image(type="pil"),
    outputs=[gr.Image(type="numpy"), gr.Textbox()],
    title="Face Recognition",
    description="Upload an image and it will show the person's name with confidence."
).launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
